In [1]:
import ee
ee.Authenticate()
ee.Initialize(project='ee-hyf7753')

*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_0JLhFqfSY1uiEaW?source=Init


In [2]:
jan_2023_climate = (
    ee.ImageCollection('ECMWF/ERA5_LAND/MONTHLY_AGGR')
    .filterDate('2023-01', '2023-02')
    .first()
)
jan_2023_climate

In [ ]:
# %pip list
# %pip install geemap
# geemap不是Google Earth Engine API的一部分，它是一个基于folium和ipywidgets的Python库，用于在Jupyter Notebook中快速创建和交互GEE地图。
import geemap

In [4]:
# geemap.set_proxy(port='7890')

In [5]:
# import os 
# os.enviorn['HTTP_PROXY']='http://127.0.0.1:7890'
# os.environ['HTTPS_PROXY'] = 'http://127.0.0.1:7890'

In [6]:
# "jupyter.widgetScriptSources": ["jsdelivr.com", "unpkg.com"],
# https://github.com/microsoft/vscode-jupyter/wiki/IPyWidget-Support-in-VS-Code-Python

In [7]:
m = geemap.Map(center=[30, 0], zoom=2)

vis_params = {
    'bands': ['temperature_2m'],
    'min': 229,
    'max': 304,
    'palette': 'inferno',
}
m.add_layer(jan_2023_climate, vis_params, 'Temperature (K)')
m

Map(center=[30, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(child…

In [8]:
cities = ee.FeatureCollection([
    ee.Feature(ee.Geometry.Point(10.75, 59.91), {'city': 'Oslo'}),
    ee.Feature(ee.Geometry.Point(-118.24, 34.05), {'city': 'Los Angeles'}),
    ee.Feature(ee.Geometry.Point(103.83, 1.33), {'city': 'Singapore'}),
])
cities

In [9]:
m.add_layer(cities, name='Cities')
m

Map(center=[30, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(child…

In [10]:
# %pip install -q --upgrade altair
import altair as alt

In [11]:
city_climates = jan_2023_climate.reduceRegions(cities, ee.Reducer.first())

city_climates_dataframe = ee.data.computeFeatures(
    {'expression': city_climates, 'fileFormat': 'PANDAS_DATAFRAME'}
)
city_climates_dataframe

,geo,city,dewpoint_temperature_2m,dewpoint_temperature_2m_max,dewpoint_temperature_2m_min,evaporation_from_bare_soil_max,evaporation_from_bare_soil_min,evaporation_from_bare_soil_sum,evaporation_from_open_water_surfaces_excluding_oceans_max,evaporation_from_open_water_surfaces_excluding_oceans_min,...,volumetric_soil_water_layer_1_min,volumetric_soil_water_layer_2,volumetric_soil_water_layer_2_max,volumetric_soil_water_layer_2_min,volumetric_soil_water_layer_3,volumetric_soil_water_layer_3_max,volumetric_soil_water_layer_3_min,volumetric_soil_water_layer_4,volumetric_soil_water_layer_4_max,volumetric_soil_water_layer_4_min
0,"{'type': 'Point', 'coordinates': [10.75, 59.91]}",Oslo,268.522069,277.052826,255.988235,2.328306e-10,-0.000028,-0.000109,6.047194e-07,-2.328306e-10,...,0.240875,0.277464,0.339737,0.242050,0.288395,0.312698,0.263550,0.283256,0.291718,0.269760
1,"{'type': 'Point', 'coordinates': [-118.24, 34....",Los Angeles,279.018688,286.388687,265.190689,2.328306e-10,-0.000191,-0.016581,1.587765e-05,-2.093231e-04,...,0.293442,0.378105,0.438995,0.267899,0.338553,0.410660,0.196136,0.217798,0.233704,0.208649
2,"{'type': 'Point', 'coordinates': [103.83, 1.33]}",Singapore,296.269420,298.315628,294.220779,0.000000e+00,-0.000365,-0.036704,1.164153e-10,-1.259167e-04,...,0.393356,0.436111,0.520004,0.405411,0.436325,0.519806,0.401047,0.496528,0.510345,0.489883


In [12]:
alt.Chart(city_climates_dataframe).mark_bar(size=100).encode(
    alt.X('city:N', sort='y', axis=alt.Axis(labelAngle=0), title='City'),
    alt.Y('temperature_2m:Q', title='Temperature (K)'),
    tooltip=[
        alt.Tooltip('city:N', title='City'),
        alt.Tooltip('temperature_2m:Q', title='Temperature (K)'),
    ],
).properties(title='January 2023 temperature for selected cities', width=500)

alt.Chart(...)